In [2]:
import pandas as pd
import numpy as np

In [2]:
listings = pd.read_csv("Beijing Airbnb Data(clean)/listings_clean.csv")

In [3]:
calendar = pd.read_csv("Beijing Airbnb Data(clean)/calendar_detail.csv")

In [4]:
reviews = pd.read_csv("Beijing Airbnb Data(clean)/reviews_detail.csv")

In [5]:
print(listings.shape,reviews.shape,calendar.shape)

(28448, 15) (202099, 6) (10384980, 7)


In [17]:
reviews=reviews[reviews.comments.str.contains(r'[^\x00-\x7F]+')== False]
reviews=reviews[reviews.reviewer_name.str.contains(r'[^\x00-\x7F]+')== False]
reviews=reviews.drop('reviewer_id', axis=1)

In [27]:
reviews

listing_id       0
id               0
date             0
reviewer_name    0
comments         0
dtype: int64

In [37]:
reviews.to_csv('reviews_final.csv',index=False)

In [20]:
listings['last_review']=listings['last_review'].replace(np.nan, 'No reviews', regex=True)
listings=listings.drop(['neighbourhood','number_of_reviews','last_review','reviews_per_month',
                        'calculated_host_listings_count'], axis=1)

In [28]:
listings.isna().sum()

id                  0
name                0
host_id             0
host_name           0
latitude            0
longitude           0
room_type           0
price               0
minimum_nights      0
availability_365    0
dtype: int64

In [36]:
listings.to_csv('listings_final.csv',index=False)

In [22]:
sample_calendar = calendar.groupby('listing_id').apply(lambda x: x.sample(frac=0.005))
sample_calendar[['price']=sample_calendar['price'].str.replace(r',', '')
sample_calendar['price']=sample_calendar['price'].str.replace(r'$', '')

AttributeError: 'DataFrame' object has no attribute 'str'

In [ ]:
sample_calendar.reset_index(drop=True, inplace=True)
sample_calendar.dropna(inplace=True)

In [ ]:
sample_calendar.head(100)

In [21]:
sample_calendar.head(100)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,44054,2019-10-31,t,$832.00,$832.00,1.0,365.0
1,44054,2019-11-06,t,$839.00,$839.00,1.0,365.0
2,100213,2019-10-15,f,$1200.00,$1200.00,1.0,30.0
3,100213,2020-03-06,f,$1320.00,$1320.00,1.0,30.0
4,128496,2019-06-15,t,$389.00,$389.00,3.0,365.0
...,...,...,...,...,...,...,...
95,1303170,2019-12-01,t,$1155.00,$1155.00,1.0,60.0
96,1325945,2019-12-23,t,$298.00,$298.00,2.0,1125.0
97,1325945,2020-03-29,t,$298.00,$298.00,2.0,1125.0
98,1370541,2019-09-18,f,$75.00,$75.00,3.0,30.0


In [18]:
sample_calendar.isna().sum()

listing_id        0
date              0
available         0
price             0
adjusted_price    0
minimum_nights    0
maximum_nights    0
dtype: int64

In [19]:
sample_calendar.to_csv('calendar_final.csv',index=False)